# curve fitting - exam preparation

\begin{aligned}
&\text { Assume you have the following market data available }\\
&\begin{array}{|c|c|c|c|c|c|}
\hline \text { EURIBOR } & \text { Fixing } & \text { FRA } & \text { Midquote } & \text { IRS } & \text { Midquote } \\
\hline 6 \mathrm{M} & 0.03478 & 1 \mathrm{X} 7 & 0.03743 & 2 \mathrm{Y} & 0.04377 \\
& & 2 \mathrm{X} 8 & 0.03871 & 3 \mathrm{Y} & 0.04625 \\
& & 3 \mathrm{X} 9 & 0.03989 & 4 \mathrm{Y} & 0.04777 \\
& & 4 \mathrm{X} 10 & 0.04098 & 5 \mathrm{Y} & 0.04875 \\
& & 5 \mathrm{X} 11 & 0.04198 & 7 \mathrm{Y} & 0.04992 \\
& & 6 \mathrm{X} 12 & 0.04289 & 10 \mathrm{Y} & 0.05081 \\
& & 7 \mathrm{X} 13 & 0.04374 & 15 \mathrm{Y} & 0.05148 \\
& & 8 \mathrm{X} 14 & 0.04452 & 20 \mathrm{Y} & 0.05181 \\
& & 9 \mathrm{X} 15 & 0.04524 & 30 \mathrm{Y} & 0.05211 \\
\hline
\end{array}
\end{aligned}

Also assume that the interest rate swaps pay $6 \mathrm{M}$ EURIBOR semiannually against a fixed rate also paid semiannually.a) Fit the zero coupon curve and compute spot rates, instantaneous forward rates and the par swap rate curve.

Note, fixing is the known points!

a) Fit the zero coupon curve and compute spot rates, instantaneous forward rates and the par swap rate curve.

In [1]:
import numpy as np
from scipy.stats import norm, ncx2, gamma
import matplotlib.pyplot as plt
import fixed_income_derivatives as fid
from scipy.optimize import minimize

In [ ]:
# reconstruct data using picture from phone
#provide type of interpolation

data = Euribor_fiing + fra_market + swap:market
interpolation_options = {"method": "linear"} # hermite degree: 3

T_cuve, R_curve, data = fid.zcb_curve_fit(data, interpolaton_options = interpolation_options)
p_curve = fid.spot_rates_to_zcb(T_curve, R_curve)

# par swap rate
R_swap_plot = np.zeros([len(T_curve)])
for i in range(1,len(R_swap_plot)):
    # maturity = T_curve[i]
    S_swap = 0
    for j in range(0,i+1):
        S_swap += 0.5*p_curve[j]
    R_swap_plot[i] = (1-p_curve[i])/S_swap

R_swap_plot # start for maturity of 1 year, since by construction value is zero for 6 month as they exchange 1 fixed for 1 floating

When plotting r_swap we cannot plot interpolated values.

The par swap rate lies almost on top of spot rates, since ZCB and swaps are not all that different since intermediate cash flows tend to net out.

In [ ]:
#p_inter, _inter, f_inter, T_inter = fid.zcb_curve_interpolation(T_fit, R_fit, interoplation_options = , resolution = 1) # 1 means data points iith montly frequence

In [ ]:
# If we want finer grid
# construct vector for which points we want interpolated balue
#T_inter = [i*1/12 for i in range(6,12,*30)]
#R_inter ? fid.interpolation
#p_inter = fid.spot_rates_to_zcb
#f_inter = fid:zcn_to_foward_rates

the forward rate curve is not at all continous, as it jumps at knot points. This hinges on the lack of differentiable spot rate curve, which was constructed using linear interpolatino, hence why we get the kinks at knot points. although zcb looks farily alright, but still room for improvement

In [2]:
## herminte is a type of polynomial and degree = 1 is linear interpolation.
# the problem with linear interpolation is. Using hermite makes it more smooth, bnut still sometjign dissatisfying around 2 yeras. But higher degree we can smooth more., maybe batter at ronft end but not as good at back end. 2 or 3 is enough!

In [ ]:
# par swap rate curve
T_swap = np.array([i for i in range(1,31)])
R_Swap, S__swap = np.zeros([len(T_swap)]), np.zeros([T_swap)])
for i, maturity in enumerate(T_swap):
    S_swap[i] = zcb_to_accrual_ractor(0,T_Swap[i], "semiannual", T_inter,p_interp)
    R_swap[i] = fid.zcb_to_par_swap_rate(0,T_swap[i],"semiannual", T_inter,p_interp)) # first entry is begining but we could let it be forward starting swap, 


the paraswap rats are smack on top of spot rates. 

## b) bump curve

we are asked to bump entry 59 (i.e. the last entry) with some amount, 0.0001 

the rate goes up, so you go up as you pay fixed, as others pay 1bps more than you!

In [ ]:
DV=1 = (R_swap-swap_armetk)*Swap # PnL

you make 15bps of the notional if entire curve is bumped 1bps

print s_wap to make sure close to duation (or simply accrual factor)

c) say you have position in 30 year swap, suppose par-swap inceases 1 bps. with accrual factor 15 the results 14 is not surprising, remember smei annually!

c) PnL

Notice, there is no 7 year swap. had it been 5 year swap, since we simply could have since it has become a 4 year swap which is in the market quote, 0.4777, there you would change i par_swap rate \times s (so you would just need to compute s). Instead we compuite par swap rate for 6 year. So compute s,R

PnL is positive 0.7%, this makes sense as our buddy entered swap at 4.8% and now the rates are 4.49%, thus he must have made money on the position since he is payer he made money. Had he sold we should change line:
"Pnl 0 (R_swap - R_pos)*S_wap" to "Pnl 0 (R_pos-R_swap)*S_wap"